In [3]:
import pandas as pd

In [4]:
courses = pd.read_csv("./static/udemy_courses.csv")

In [5]:
courses['num_subscribers'].describe()

count      3678.000000
mean       3197.150625
std        9504.117010
min           0.000000
25%         111.000000
50%         911.500000
75%        2546.000000
max      268923.000000
Name: num_subscribers, dtype: float64

In [6]:
pop_cou = courses[(courses['num_subscribers']>12000) & (courses['num_reviews']>5000)]

In [7]:
pop_cou.head(2)

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
1979,238934,Pianoforall - Incredible New Way To Learn Pian...,https://www.udemy.com/pianoforall-incredible-n...,True,200,75499,7676,362,All Levels,30.0,2014-08-07T06:27:51Z,Musical Instruments
2589,473160,Web Design for Web Developers: Build Beautiful...,https://www.udemy.com/web-design-secrets/,False,0,98867,6512,20,All Levels,3.0,2015-04-13T18:29:47Z,Web Development


In [11]:
courses['tags'] = courses['course_title'] + " " + courses['subject']

In [9]:
courses[courses['course_id']==625204]

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject
3230,625204,The Web Developer Bootcamp,https://www.udemy.com/the-web-developer-bootcamp/,True,200,121584,27445,342,All Levels,43.0,2015-11-02T21:13:27Z,Web Development


In [29]:
courses['url'].values[10]

'https://www.udemy.com/trading-with-amazons-aws-servers-trade-secrets-of-the-pros/'

In [19]:
from bs4 import BeautifulSoup
import requests

In [30]:
response = requests.get(courses['url'].values[10])

output = []
if response.status_code == 200:
    bs = BeautifulSoup(response.content, 'html.parser')
    spans = bs.find("span", class_="intro-asset--img-aspect--3gluH")
    if spans:
        first_image = spans.find("img")
        output.append(first_image.get("src"))
        
    print(output)

['https://img-c.udemycdn.com/course/240x135/592338_1537_4.jpg']


In [49]:
pop_cou.sort_values(by='num_subscribers', ascending=False, ignore_index=True).shape

(16, 12)

In [41]:
courses['num_reviews'].describe()

count     3678.000000
mean       156.259108
std        935.452044
min          0.000000
25%          4.000000
50%         18.000000
75%         67.000000
max      27445.000000
Name: num_reviews, dtype: float64

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [13]:
tokemizer = TfidfVectorizer().fit_transform(courses['tags'])
similarity = cosine_similarity(tokemizer)

In [25]:
def content_based_filtering(c_id, n_recommend):
    n_recommend = int(n_recommend)
    course_index = courses[courses['course_id'] == c_id].index[0]
    distances = similarity[course_index]
    course_lists = sorted(list(enumerate(distances)), reverse=True, key=lambda x:x[1])[1:n_recommend]

    for i in course_lists:
            print(courses.iloc[i[0]])

In [26]:
content_based_filtering(625204, 5)

course_id                                                         390866
course_title           The Complete Web Developer Bootcamp - Beginner...
url                       https://www.udemy.com/job-ready-web-developer/
is_paid                                                             True
price                                                                190
num_subscribers                                                    14103
num_reviews                                                         1548
num_lectures                                                         197
level                                                         All Levels
content_duration                                                    21.5
published_timestamp                                 2015-01-19T00:23:15Z
subject                                                  Web Development
tags                   The Complete Web Developer Bootcamp - Beginner...
Name: 2696, dtype: object
course_id                